In [ ]:
# Import functions
import numpy as np
import matplotlib.pyplot as p
% matplotlib inline

import eigencurves
import eigenmaps
import kmeans
#import mapPCA
import bin_eigenspectra
import os
import pdb

import gen_lightcurves
import healpy

import run_higher_sph_harm

from importlib import import_module
planet_name = 'HD189733b'
#system = import_module('data.planet.{}'.format(planet_name))

### Import spectra and generate map

In [ ]:
# ...

### Generate lightcurve using STARRY

In [ ]:
# ...

## Get the noisy lightcurves
This is not used because it assumes you already ran `run_higher_sph_harm.py`

In [ ]:
#from eigensource import add_noise

In [ ]:
#inputLC3D = add_noise.get_lc(inputFile='data/input_lightcurves/eclipse_lightcurve_test1.npz')
#noiseDict = add_noise.add_noise(inputLC3D)

### Fit eigencurves to lightcurve

In [ ]:
## Run higher spherical harmonics
## Results are saved in data/sph_harmonic_coefficients_full_samples
for oneOrd in np.arange(2,6+1):
    run_higher_sph_harm.run_lc_noise_and_fit(norder=oneOrd)


### Turn SH coefficients into maps

In [ ]:
def retrieve_map_full_samples(degree=3,testNum=1):
    dataDir = "data/sph_harmonic_coefficients_full_samples/eclipse_lightcurve_test{}/".format(testNum)
    tmp = np.load("{}spherearray_deg_{}.npz".format(dataDir,degree))
    outDictionary = tmp['arr_0'].tolist()
    
    londim = 100
    latdim = 100
    samples = outDictionary['spherical coefficients'] # output from eigencurves
    waves = outDictionary['wavelength (um)']
    
    randomIndices = np.random.randint(0,len(samples),99)
    nRandom = len(randomIndices)
    
    fullMapArray = np.zeros([nRandom,len(waves),londim,latdim])
    for drawInd, draw in enumerate(samples[randomIndices]):
        inputArr = np.zeros([len(waves),samples.shape[1]+1])
        inputArr[:,0] = waves
        inputArr[:,1:] = draw.transpose()
        
        wavelengths, lats, lons, maps = eigenmaps.generate_maps(inputArr,
                                                                N_lon=londim, N_lat=latdim)
        fullMapArray[drawInd,:,:,:] = maps
    
    return fullMapArray, lats, lons
    


In [ ]:
def plot_retrieved_map(fullMapArray,lats,lons,waveInd=3,degree=3):
    percentiles = [5,50,95]
    mapLowMedHigh = np.percentile(fullMapArray,percentiles,axis=0)
    #medianMap = np.median(fullMapArray,axis=0)
    londim = fullMapArray.shape[2]
    
    fig, axArr = p.subplots(1,3,figsize=(22,5))
    for ind,onePercentile in enumerate(percentiles):
        map_day = mapLowMedHigh[ind][waveInd][:,londim//4:-londim//4]
        extent = np.array([np.min(lons),np.max(lons),np.min(lats),np.max(lats)])/2./np.pi*180
        plotData = axArr[ind].imshow(map_day, extent=extent)
        cbar = fig.colorbar(plotData,ax=axArr[ind])
        cbar.set_label('Brightness')
        axArr[ind].set_ylabel('Latitude')
        axArr[ind].set_xlabel('Longitude')
        axArr[ind].set_title("{} %".format(onePercentile))
        #axArr[ind].show()
    
    fig.suptitle('Retrieved group map, n={}, {:.2f}$\mu$m'.format(degree,waves[waveInd]))

def get_map_and_plot(waveInd=3,degree=3,testNum=1):
    fullMapArray, lats, lons = retrieve_map_full_samples(degree=degree,testNum=testNum)
    plot_retrieved_map(fullMapArray,lats,lons,degree=degree,waveInd=waveInd)

def all_sph_degrees(waveInd=5):
    for oneDegree in np.arange(2,6+1):
        get_map_and_plot(waveInd=waveInd,degree=oneDegree)

### Show the original Map

In [ ]:
def show_orig_map(waveInd=0,testNum=1):
    """
    Show the original map at a given wavelength
    
    Parameters
    -----------
    waveInd: int
        The wavelength index
    testNum: int
        The test Number (ie. lightcurve number)
    """
    origData = np.load("data/maps/mystery_map{}.npz".format(testNum))
    lammin1 = 2.41; lammax1 = 3.98; dlam1 = 0.18
    spaxels = origData["spaxels"]
    lam = origData["wl"]
    lamlo, dlamlo = gen_lightcurves.construct_lam(lammin1, lammax1, dlam=dlam1)
    Nlamlo = len(lamlo)

    # Set HealPy pixel numbers
    Npix = spaxels.shape[0]

    # Define empty 2d array for spaxels
    spec2d = np.zeros((Npix, Nlamlo))

    # Loop over pixels filling with spectra
    for i in range(Npix):
        # Degrade the spectra to lower resolution
        spec2d[i,:] = gen_lightcurves.downbin_spec(spaxels[i, :], lam, lamlo, dlam = dlamlo)


    healpy.mollview(spec2d[:,waveInd], title=r"%0.2f $\mu$m" %lamlo[waveInd])
    p.show()
    return spec2d

## Spot check retrieved maps against original for the Quadrant Test Case

In [ ]:
spec2d = show_orig_map(waveInd=1)

### Show the posteriors for the retrieved maps

In [ ]:
all_sph_degrees(waveInd=1)

In [ ]:
spec2D = show_orig_map(5)

In [ ]:
all_sph_degrees(5)

## Loop over many samples to get eigenmap and eigenspectra Errors

In [ ]:
def find_groups(ngroups=4,degree=2,testNum=1,
               trySamples=45):
    """ 
    Find the eigenspectra using k means clustering
    
    Parameters
    ----------
    ngroups: int
        Number of eigenspectra to group results into
    degree: int
        Spherical harmonic degree to draw samples from
    testNum: int
        Test number (ie. lightcurve number 1,2, etc.)
    trySamples: int
        Number of samples to find groups with
        All samples take a long time so this takes a random
        subset of samples from which to draw posteriors
    """
    samplesDir = "data/sph_harmonic_coefficients_full_samples"
    dataDir = "{}/eclipse_lightcurve_test{}/".format(samplesDir,testNum)
    tmp = np.load("{}spherearray_deg_{}.npz".format(dataDir,degree))
    outDictionary = tmp['arr_0'].tolist()
    samples = outDictionary['spherical coefficients'] # output from eigencurves

    eigenspectra_draws = []
    kgroup_draws = []

    randomIndices = np.random.randint(0,len(samples),trySamples)
    for draw in samples[randomIndices]:
        ## Re-formatting here into a legacy system
        ## 1st dimension is wavelength
        ## 2nd dimensions is data (0th element = wavelength)
        ##                        (1: elements are spherical harmonic coefficients)
        inputArr = np.zeros([len(waves),samples.shape[1]+1])
        inputArr[:,0] = waves
        inputArr[:,1:] = draw.transpose()

        wavelengths, lats, lons, maps = eigenmaps.generate_maps(inputArr, N_lon=100, N_lat=100)

        kgroups = kmeans.kmeans(maps, ngroups)

        eigenspectra = bin_eigenspectra.bin_eigenspectra(maps, kgroups)

        eigenspectra_draws.append(eigenspectra)
        kgroup_draws.append(kgroups)
    return eigenspectra_draws, kgroup_draws

In [ ]:
eigenspectra_draws, kgroup_draws = find_groups(degree=5)

In [ ]:
for oneSpec in np.array(eigenspectra_draws):
    p.plot(outDictionary['wavelength (um)'],oneSpec[0,:])
p.xlabel('Wavelength ($\mu$m)')
p.ylabel('F$_p$/F$_*$')

In [ ]:
eigenspectra = np.mean(eigenspectra_draws, axis=0)
eigenerrs = np.std(eigenspectra_draws, axis=0)

kgroups = np.mean(kgroup_draws, axis=0)

In [ ]:
waves = outDictionary['wavelength (um)']
for spec, err in zip(eigenspectra, eigenerrs):
    p.errorbar(waves, spec, err)
p.xlabel('Wavelength (micron)')
p.ylabel('Fp/Fs')
p.title('Eigenspectra from light-curve fit')
p.show()

In [ ]:
def show_eigenspec_and_map(ngroups=4,degree=5):
    """ Show the eigenspectra and map"""
    fig, (ax0, ax1) = p.subplots(1,2,figsize=(12,4))
    for ind,spec, err in zip(range(ngroups), eigenspectra, eigenerrs):
        ax0.errorbar(waves, spec, err, marker='o',
                  color=p.cm.viridis(float(ind)/float(ngroups-1)))
    ax0.set_xlabel('Wavelength (micron)')
    ax0.set_ylabel('Fp/Fs')
    ax0.set_title('Eigenspectra from light-curve fit, n={}'.format(degree))

    imData = ax1.imshow(kgroups)
    cbar = fig.colorbar(imData, ticks=np.arange(ngroups),ax=ax1)
    cbar.set_label('# Group')
    ax1.set_xlabel('Latitude')
    ax1.set_ylabel('Longitude')
    ax1.set_title('Retrieved group map')

In [ ]:
show_eigenspec_and_map()

## Show the original spectra and map

In [ ]:
uniqueSpec, uniqueMap = np.unique(spec2d,axis=0,return_inverse=True)
nUniqueSpec = uniqueSpec.shape[0]
groupSpec2D = np.zeros(spec2d.shape[0])
fig, ax0 = p.subplots()
for ind,oneSpec in enumerate(uniqueSpec):
    color=p.cm.viridis(float(ind)/float(nUniqueSpec-1))
    ax0.plot(noiseDict['wavelength (um)'],oneSpec,color=color,label="Spec {}".format(ind))
ax0.legend()
ax0.set_xlabel('Wavelength ($\mu$m)')
ax0.set_ylabel('Relative Flux')
fig.savefig('plots/original_maps/orig_quadrant_spec.pdf')
plotObj = healpy.mollview(uniqueMap,title='Spectral map',unit='Spectrum #')
p.savefig('plots/original_maps/orig_quadrant_map.pdf')

# The Hot Spot Model

In [ ]:
eigenspectra_draws, kgroup_draws = find_groups(degree=3,testNum=2,
                                              trySamples=200,ngroups=3)

In [ ]:
for oneSpec in np.array(eigenspectra_draws):
    p.plot(outDictionary['wavelength (um)'],oneSpec[0,:],alpha=0.05,
          color='blue')
p.xlabel('Wavelength ($\mu$m)')
p.ylabel('F$_p$/F$_*$')
p.savefig('plots/eigenmap_and_spec/hot_spot_spectra_deg3_3groups.pdf',
         bbox_inches='tight')

In [ ]:
eigenspectra = np.mean(eigenspectra_draws, axis=0)
eigenerrs = np.std(eigenspectra_draws, axis=0)

kgroups = np.mean(kgroup_draws, axis=0)

In [ ]:
waves = outDictionary['wavelength (um)']
for spec, err in zip(eigenspectra, eigenerrs):
    p.errorbar(waves, spec, err)
p.xlabel('Wavelength (micron)')
p.ylabel('Fp/Fs')
p.title('Eigenspectra from light-curve fit')
#p.show()
p.savefig('plots/eigenmap_and_spec/'+
          'hot_spot_spectra_deg3_3groups_error_bars.pdf',
         bbox_inches='tight')